### Project Dashboard developed in Bokeh

run in terminal from root directory<p>

```
bokeh serve --show project_dashboard.ipynb
```

In [30]:
from bokeh.models import (ColorBar, ColumnDataSource,
                          GeoJSONDataSource, HoverTool,Range1d,
                          LinearColorMapper, Slider, Select, Column, Text, Plot,TapTool)

from bokeh.plotting import figure

from bokeh.layouts import column, row, layout, grid
from bokeh.tile_providers import CARTODBPOSITRON, get_provider
from bokeh.io import curdoc, show, output_file

from bokeh.palettes import brewer


import json
import geopandas as gpd
import pandas as pd
import numpy as np

import logging

from chart_constants import FONT_PROPS_LG, FONT_PROPS_SM, FONT_PROPS_MD,  PLOT_FORMATS

In [4]:
%load_ext watermark
%watermark --iversions

The watermark extension is already loaded. To reload it, use:
  %reload_ext watermark
logging  : 0.5.1.2
numpy    : 1.21.5
geopandas: 0.10.2
pandas   : 1.3.5
json     : 2.0.9
sys      : 3.8.10 (default, Nov 26 2021, 20:14:08) 
[GCC 9.3.0]



In [5]:
# loading chemichal data dataset 
chem_data = pd.read_csv(r'assets/chem_data.csv', parse_dates=['VisitDate'])
chem_data['year'] = chem_data.VisitDate.dt.year

seasons = {
    '0':'winter',
    '1':'spring',
    '2':'summer',
    '3':'fall'
}

chem_data['season'] =  (chem_data.VisitDate.dt.month - 1) //3
chem_data['season'] = chem_data.season.apply(lambda x: seasons[str(x)])
chem_data['date'] = chem_data['VisitDate'].apply(lambda x:x.strftime("%Y-%m-01"))

In [6]:
# Since there are multiple measurments for each year but
#  mainly in spring and summer it makes sense to take the mean value for that year

df = chem_data.groupby(['LakeID', 'year', 'CharacteristicID'])['Result'].agg(np.mean)
df = df.reset_index()

In [29]:
# Loading of the geoJson data for geographuical plotting
gdf = gpd.read_file(r'assets/Lakes_Inventory.geojson')

# Check projection
gdf.crs

<Geographic 2D CRS: EPSG:4326>
Name: WGS 84
Axis Info [ellipsoidal]:
- Lat[north]: Geodetic latitude (degree)
- Lon[east]: Geodetic longitude (degree)
Area of Use:
- name: World.
- bounds: (-180.0, -90.0, 180.0, 90.0)
Datum: World Geodetic System 1984 ensemble
- Ellipsoid: WGS 84
- Prime Meridian: Greenwich

In [8]:
# Reproject to a projection that fits the projection of the tiles
gdf.geometry = gdf.geometry.to_crs('World_Mercator')
gdf.crs

<Derived Projected CRS: ESRI:54004>
Name: World_Mercator
Axis Info [cartesian]:
- E[east]: Easting (metre)
- N[north]: Northing (metre)
Area of Use:
- name: World.
- bounds: (-180.0, -90.0, 180.0, 90.0)
Coordinate Operation:
- name: World_Mercator
- method: Mercator (variant B)
Datum: World Geodetic System 1984
- Ellipsoid: WGS 84
- Prime Meridian: Greenwich

In [26]:
# get data for lake health metric

metric_df = pd.read_csv('assets/health_metric.csv', index_col='Lake')
metric_df = metric_df[[ 'Health_Score']]

metric_df.reset_index(inplace=True)

metric_df = metric_df.merge(chem_data, how='outer', left_on='Lake', right_on='LakeID')
metric_df = metric_df[['Health_Score', 'LakeID']].drop_duplicates()
metric_df

,Health_Score,LakeID
0,1.124186,RESCUE
3661,0.439404,METCALF
4429,0.425024,BURR (SUDBRY)
4884,0.169875,GREEN RIVER
6382,1.315273,WOODWARD
...,...,...
281731,NaN,WHEELER (BRUNWK)
282382,NaN,WINONA
282506,NaN,WOLCOTT
282908,NaN,WRIGHTSVILLE


In [9]:
#Functions for updating data to be displayed in the dashboard


def column_data(selectedYear,CharacteristicID):
    yr = selectedYear
    df_yr = df[(df['year'] == yr) & (df['CharacteristicID'] == CharacteristicID)]
    
    result_min = df[df.CharacteristicID == CharacteristicID].Result.min()
    result_max = df[df.CharacteristicID == CharacteristicID].Result.max()
    
    merged = gdf.merge(df_yr, left_on='LAKEID', right_on='LakeID', how = 'left')
    merged.fillna('No data', inplace = True)
    
    return data, result_min, result_max


def json_data(selectedYear,CharacteristicID):
    
    yr = selectedYear
    df_yr = df[(df['year'] == yr) & (df['CharacteristicID'] == CharacteristicID)]
    
    result_min = df[df.CharacteristicID == CharacteristicID].Result.min()
    result_max = df[df.CharacteristicID == CharacteristicID].Result.max()
    
    merged = gdf.merge(df_yr, left_on='LAKEID', right_on='LakeID', how = 'left')
    merged.fillna('No data', inplace = True)


    merged_json = json.loads(merged.to_json())
    json_data = json.dumps(merged_json)


    return json_data, result_min, result_max

def time_series_data(CharacteristicID, index):
 
    LakeID = gdf.iloc[index].LAKEID
    df_Lake = df[(df['CharacteristicID'] == CharacteristicID) & (df['LakeID'] == LakeID)]

    return df_Lake

def time_series_LakeID(index):
    return gdf.iloc[index].LAKEID

def lake_health(index):
    LakeID =  gdf.iloc[index].LAKEID

    temp_metric = metric_df[metric_df.LakeID == LakeID]

    return temp_metric

In [32]:
# Initialize source data for the geographical map 
geosource=GeoJSONDataSource(geojson=json_data(1980, "TP")[0])

# Initialize the source data for time series scatter plot
source=ColumnDataSource(time_series_data("TP", 0))

metric_source = ColumnDataSource(lake_health(0))

# Define a sequential multi-hue color palette.
palette = brewer["YlGnBu"][8]

# Reverse color order so that dark blue is highest obesity.
palette = palette[::-1]

# Instantiate LinearColorMapper that linearly maps numbers in a range, into a sequence of colors. Input nan_color.
color_mapper = LinearColorMapper(palette = palette, low = 0, high = 40, nan_color = "#d9d9d9")

# Create color bar.
color_bar = ColorBar(color_mapper=color_mapper, label_standoff=8,width = 20, height = 550,
                    border_line_color=None, location = (0,0), orientation = "vertical")

# Add hover tool
hover = HoverTool(tooltips = [ ("Lake","@LakeID"),("Result", "@Result")])

# Add tap-tool
tap = TapTool(behavior='select', gesture='tap')


# create the header row for the dashboard

xdr = Range1d(0, 220)                                                       
ydr = Range1d(0, 120)

header = Plot(
    x_range=xdr,                                                            
    y_range=ydr,                                                                                                       
    title="",                                                               
    plot_width=1100,                                                         
    plot_height=100,                                                        
    min_border=0,
    background_fill_color='#9fbe25',
    background_fill_alpha=0.8,    
    **PLOT_FORMATS
                                                           
)

title = Text(x=30, y=30, text=['VERMONT LAKE MONITORING PROGRAMM'], text_font={'value':'montserrat'},
    text_color='white',
    text_font_style="bold",
    text_font_size='23pt',
    )
header.add_glyph(title)

# Create a text object to display leake health metric
                                                            
                                                       

plot = Plot(                                                                
    x_range=xdr,                                                            
    y_range=ydr,                                                            
    title="",                                                               
    plot_width=600,                                                         
    plot_height=200,                                                        
    min_border=0,
    **PLOT_FORMATS
                                                           
)

explanation = ["We put forward the following metric for lake health.\n \
We look at mean total phosphorus, chlorophyl-a, and Secchi water clarity scores,\n \
for each lake.  Then we look at each lake's z-score  (in absolute value)\n \
This metric measures (1) how far a lake's mean measures are from the other lakes,\n \
and (2) to what degree the trend in these measures diverges from that in other lakes."]

# Add the writing                                                           
metric_title = Text(x=0, y=90, text=['Lake Health metric'], text_font_style="normal",
    text_font_size='23pt')

metric_explanation = Text(x=0, y=40, text=explanation, text_font_style="normal",
    text_font_size='10pt')

metric_value = Text(x=0,y=0, text='@Health_Score', text_font_style="normal",
    text_font_size='18pt')
    
plot.add_glyph(metric_title)
plot.add_glyph(metric_source, Text(), selection_glyph=metric_value)
plot.add_glyph(metric_explanation)

# Create a figure object for line chart

time_series = figure(title = 'Chemichal data time-series', 
           plot_height = 200,
           plot_width = 600, 
           toolbar_location = None)
time_series.xgrid.visible = False
time_series.ygrid.visible = False
time_series.xaxis.major_tick_line_color = 'grey'  
time_series.xaxis.major_tick_line_color = None
time_series.yaxis.minor_tick_line_color = None
time_series.yaxis.major_tick_line_color = 'grey'
time_series.yaxis.major_label_text_font_size = '10pt'  
time_series.xaxis.major_label_text_font_size = '10pt' 
time_series.yaxis.major_tick_in = -6
time_series.xaxis.minor_tick_in = -6
time_series.x_range = Range1d(1979, 2022)
time_series.outline_line_color=None

# Add scatter plot renderer time_series figure
time_series.scatter(source = source, x='year', y='Result', line_color = None, fill_alpha=0.5, color='green' )


# Create figure object for geomap.

tile_provider = get_provider(CARTODBPOSITRON)

geo = figure(title = "Vermont Lay monitoring programm",
           plot_height = 600,
           plot_width = 500,
           toolbar_location = 'left',
           tools = [hover, tap, 'pan', 'wheel_zoom'],
           x_axis_location=None,
           y_axis_location=None,
           x_range=(-6500000, -10000000), y_range=(5000000, 6000000),
            x_axis_type="mercator", y_axis_type="mercator"
          )


geo.xgrid.visible = False
geo.ygrid.visible = False
geo.outline_line_color=None
geo.add_tile(tile_provider)

# Add patch renderer to figure.
geo.patches("xs","ys",
          source = geosource,
          fill_color = {"field" :"Result",
                        "transform" : color_mapper},
          line_color = "black",
          line_width = 0.25,
          fill_alpha = 1)



# Define the callback function: update_plot
def update_plot(attr, old, new):
    
    yr = slider.value
    CharacteristicID = parameter_select.value
    
    new_data, color_mapper.low, color_mapper.high = json_data(yr, CharacteristicID)
    
    geosource.geojson = new_data
    geo.title.text = "Vermont Lay monitoring programm, {} Parammeter: {}".format(yr,CharacteristicID)
    
    

# Define a callback function: update_time_series
def update_time_series(attr, old, new):
    
    selections = new
    CharacteristicID = parameter_select.value
    log = logging.getLogger('bokeh')
    log.info(selections)
    
    new_data = time_series_data(CharacteristicID, selections[0])
    source.data = new_data
    LakeID = time_series_LakeID(selections[0])
    log.info(LakeID)
    log.info(type(LakeID))
    time_series.title.text = "Lake {} {} data time-series".format(LakeID, CharacteristicID)
    

def update_metric(attr, old, new):
    selections = new

    metric_data = lake_health(selections[0])
    metric_source.data = metric_data
    log = logging.getLogger('bokeh')
    log.info(selections)
    log.info(metric_data)
    log.info(type(metric_data))



# Make a slider object: slider 
slider = Slider(title = "Year",start = 1980, end = 2021, step = 1, value = 1980, width=500)
slider.on_change("value",update_plot)

# Make a drop down object: selector 
LABELS = list(df.CharacteristicID.unique())
parameter_select = Select(value="TP", options=LABELS)
parameter_select.on_change("value", update_plot)
    
    
geosource.selected.on_change('indices', update_time_series, update_metric)




#Specify layout
geo.add_layout(color_bar, "left")


#first_column = column(geo, Column(slider, parameter_select))
#second_column = column(time_series, plot)




# Make a column layout of widgetbox(slider) and plot, and add it to the current document
l = layout([
[header],
[geo, column(time_series, plot)],
[Column(slider, parameter_select)],
])
curdoc().add_root(l)

NameError: name 'lake_health' is not defined